In [1]:
import pandas as pd
import re
import json
import xml.etree.ElementTree as xet
import requests as req
import time
import glob
import csv
import regex
import tqdm
import pytz
import numpy as np
import io
from bs4 import BeautifulSoup

In [2]:
def parse_sitemap(fn):

    xml = xet.parse(fn)
    xml_root = xml.getroot()
    news_list = []
    
    for news in xml_root:
        try:
            result = {}
            result['url'] = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}loc').text
            lastmod = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}lastmod')
            if lastmod is not None:
                result['lastmod'] = lastmod.text

            #title = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}news:title').text
            #keywords = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}news:keywords').text
            #news_list.append(dict(zip(['url', 'lastmod'], [url, lastmod])))
            news_list.append(result)
        except:
            continue

    return news_list

In [3]:
sitemaps = parse_sitemap(io.StringIO(req.get('https://zona.media/sitemap.xml').content.decode()))
sitemaps_filtered = []
for sitemap in sitemaps:
    if re.search(r'2020|2019', sitemap['url']):
        sitemaps_filtered.append(sitemap)

In [4]:
news_db = []

for fn in tqdm.tqdm_notebook(sitemaps_filtered):
    s1 = time.time()

    try:
        text = io.StringIO(req.get(fn['url']).content.decode())
        news = parse_sitemap(text)
    except Exception as e:
        print(e)
        continue
        
    news_db += news
    s2 = time.time()
    time.sleep(max(1 - s2 + s1, 0))

<ipython-input-4-73c316e576b5>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for fn in tqdm.tqdm_notebook(sitemaps_filtered):


In [5]:
df = pd.DataFrame(news_db)
df['lastmod'] = pd.to_datetime(pd.Series(df.url.str.extract(r'(\d{4,4}\/\d+\/\d+)').values.ravel()), format='%Y/%m/%d', utc=True)
df.head()

,url,lastmod
0,https://zona.media/news/2020/12/13/girl,2020-12-13 00:00:00+00:00
1,https://zona.media/news/2020/12/13/vnkmt,2020-12-13 00:00:00+00:00
2,https://zona.media/news/2020/12/13/bunker,2020-12-13 00:00:00+00:00
3,https://zona.media/news/2020/12/13/putin,2020-12-13 00:00:00+00:00
4,https://zona.media/chronicle/koronavirus_dcmbr,NaT


In [6]:
df[(df.lastmod >= '2019-11-01')].to_json('news_mediazona.json', date_unit='s', date_format='iso')
df[(df.lastmod >= '2019-11-01')].shape
#df[(df.lastmod >= '2018-01-01') & (df.lastmod < '2019-07-01')].shape

(7125, 2)

# Parsing

In [30]:
def parse_news(text):
    soup = BeautifulSoup(text, features='lxml')
    
    news_text = ''
    news_body = soup.find(name='div', attrs={'class': 'mz-publish__wrapper'})
    
    if news_body:
        bad = news_body.find_all(name=['script', 'iframe', 'svg'])
        if bad:
            [tag.decompose() for tag in bad]
        
        news_ps = news_body.find_all(name=['p', 'h2', 'li'])
        if news_ps:
            news_text += " " + " ".join([news_p.get_text(" ", strip=True) for news_p in news_ps])
        news_text = re.sub(r'[\n\r]+', ' ', news_text)
    
    
    description_elem = soup.find(name="meta", attrs={'name': "description"})
    description = re.sub(r'[\n\r]+|<[^>]+>', ' ', description_elem.get('content')) if description_elem else ''
        
    return {'news_text': news_text, 'description': description}

In [31]:
parse_news(req.get(df.iloc[-1].url).content)

{'news_text': ' В больницу с обморожением доставили новосибирского рабочего, которые трое суток провел на площадке строительного крана, требуя выплаты зарплаты. С крана мужчину сняли спасатели, пишет «МБХ-медиа». Работавший на стройке новосибирец забрался на 90-метровый кран утром 28 декабря. «Долг по зарплате у него 140 тысяч — за март, за апрель, сентябрь и август. Сегодня свет отрубили на кране, чтобы слез, но он не собирается, ему нечего терять, семьи у него нет. Никто не реагирует на него», — рассказывал «НГС.Новости» коллега строителя. Работодатель новосибирца — застройщик «ПТК-30». Сотрудники этой компании уже не первый раз требуют погасить долги по зарплате: двое строителей требовали выплатить им зарплату в августе, в октябре на башенный кран забрались десять человек, в декабре — трое . Раз в неделю наши авторы делятся своими впечатлениями от главных событий и текстов',
 'description': 'В больницу с обморожением доставили новосибирского рабочего, которые трое суток провел на пл